# Load Video Frames

In [1]:
# Need opencv and cap_from_youtube installed
# In anaconda env in terminal: 
#   pip install cap_from_youtube
#   conda install -c conda-forge opencv
# cap_from_youtube gets youtube video without downloading
import cv2
from cap_from_youtube import cap_from_youtube

In [15]:
# User input:
url = "https://www.youtube.com/watch?v=262hmHGMANE"
#####################################################

# Returns VideoCapture object
capture = cap_from_youtube(url, '144p')

# Get number of frames and frames per second
num_frames = capture.get(cv2.CAP_PROP_FRAME_COUNT)
fps = capture.get(cv2.CAP_PROP_FPS)

frames = []

frame_count = 0
while True:
    ret, frame = capture.read()
    if ret:
        #cv2.imshow('video', frame)
        # waitKey miliseconds between frames defines how fast you can see video!!
        #cv2.waitKey(1)
        frames.append(frame)
    if frame_count == num_frames:
        break
    frame_count += 1
capture.release()
#cv2.destroyAllWindows()

[youtube] Extracting URL: https://www.youtube.com/watch?v=262hmHGMANE
[youtube] 262hmHGMANE: Downloading webpage
[youtube] 262hmHGMANE: Downloading android player API JSON


In [3]:
# Save frames if you don't want to re-load video
# Large file can't be pushed to git though
import pickle
with open("frames", "wb") as fp:   #Pickling
    pickle.dump(frames, fp)
with open("frames", "rb") as fp:   # Unpickling
    frames_loaded = pickle.load(fp)

In [9]:
# Check frames was saved and loaded correctly
import numpy as np
frames = np.array(frames)
frames_loaded = np.array(frames_loaded)
if np.array_equal(frames,frames_loaded):
    print('Frames was correctly saved and loaded')

Frames was correctly saved and loaded


<a href="https://colab.research.google.com/github/haoyuwu03/dancify/blob/main/Dancify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# import the necessary packages
from imutils.video import VideoStream
import argparse
import datetime
import imutils
import time
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
# construct the argument parser and parse the arguments
#ap = argparse.ArgumentParser()
#ap.add_argument("-v", "--video", help="drive/MyDrive/Videos/dance.MOV")
#ap.add_argument("-a", "--min-area", type=int, default=500, help="minimum area size")
#args = vars(ap.parse_args())

#reading from a video file
vs = cv2.VideoCapture("drive/MyDrive/newDingga.mp4")
# initialize the first frame in the video stream
firstFrame = vs.read()[1]
frameWidth = firstFrame.shape[1]
frameHeight = firstFrame.shape[0]
# Change rate to slow down or speed up video (1=full speed)
rate = 1
fps = 30*rate # 30 fps is full speed
vid_writer = cv2.VideoWriter('drive/MyDrive/dinggaDancify.mp4',cv2.VideoWriter_fourcc(*'MP4V'), fps, (frameWidth,frameHeight))

In [ ]:
# Specify the paths for the 2 files
protoFile = "drive/MyDrive/Scale and Coin/models/pose/coco/pose_deploy_linevec.prototxt"
weightsFile = "drive/MyDrive/Scale and Coin/models/pose/coco/pose_iter_440000.caffemodel"
# Read the network into Memory
net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)
# Using MPI model
nPoints = 18
# Specify the input image dimensions
inWidth = 368
inHeight = 368
# Specify the threshold
threshold = 0.01
# Point pairs to draw line between for skeleton
#Right Shoulder – 2, Right Elbow – 3, Right Wrist – 4, Left Shoulder – 5, Left Elbow – 6, Left Wrist – 7, 
#Right Hip – 8, Right Knee – 9, Right Ankle – 10, Left Hip – 11, Left Knee – 12, LAnkle – 13
POSE_PAIRS = [ [1,0],[0,14],[0,15],[14,16],[15,17]]
RIGHT_POSE_PAIRS = [[1,2],[2,3],[3,4],[1,8],[8,9],[9,10]]
LEFT_POSE_PAIRS = [[1,5],[5,6],[6,7],[1,11],[11,12],[12,13]]

In [ ]:
frames_before_update = 6 #number of frames before calculating new points
frameCount = frames_before_update
maxFrames = 5000
count = 0
last = -1
# List to store detected keypoints
points = []
# loop over the frames of the video
while True:
    if count <= last:
        continue
    if count >= maxFrames:
        break
    count += 1
    frameCount+=1
    # grab the current frame
    frame = vs.read()
    frame = frame[1]
    frameCopy = np.copy(frame)
    # if the frame could not be grabbed, then we have reached the end
	# of the video
    if frame is None:
        break
    if frameCount > frames_before_update:
        frameCount = 0;
        # Clear points to store new points
        points = []
        # Prepare the frame to be fed to the network
        inpBlob = cv2.dnn.blobFromImage(frame, 1.0 / 255, (inWidth, inHeight),
                                    (0,0,0), swapRB=False, crop=False)
        # Set the prepared object as the input blob of the network
        net.setInput(inpBlob)

        # Make predictions on where the body joints are
        output = net.forward()

        # Get height and width of prediction
        H = output.shape[2]
        W = output.shape[3]

        for i in range(nPoints):
            # confidence map of corresponding joint location
            probMap = output[0, i, :, :]

            # Find global maxima of the probMap
            minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)

            # Scale the point to fit on the original image
            x = (frameWidth * point[0]) / W
            y = (frameHeight * point[1]) / H

            # If the probability is greater than the threshold, draw the point and add to list
            if prob > threshold:
                # CHANGE THESE LINES TO DESIGN POINTS
                cv2.circle(frameCopy, (int(x),int(y)), 8, (0, 255, 255), thickness=1, 
                       lineType=cv2.FILLED)
                cv2.putText(frameCopy, "{}".format(i), (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 
                        1, (0, 0, 255), 2, lineType=cv2.LINE_AA)
                points.append((int(x), int(y)))

            else:
                points.append(None)
    # Draw skeleton between center points
    """for pair in POSE_PAIRS:
        partA = pair[0]
        partB = pair[1]

        if points[partA] and points[partB]:
            # CHANGE THESE LINES TO DESIGN SKELETON
            cv2.line(frame, points[partA], points[partB], (224, 224, 224), 3, lineType=cv2.LINE_AA)
            cv2.circle(frame, points[partA], 8, (178, 102, 255), thickness=-1, lineType=cv2.FILLED)
            cv2.circle(frame, points[partB], 8, (178, 102, 255), thickness=-1, lineType=cv2.FILLED)"""
    # Draw skeleton between right points
    for pair in RIGHT_POSE_PAIRS:
        partA = pair[0]
        partB = pair[1]

        if points[partA] and points[partB]:
            # CHANGE THESE LINES TO DESIGN SKELETON
            cv2.line(frame, points[partA], points[partB], (255, 204, 153), 30, lineType=cv2.LINE_AA)
            cv2.circle(frame, points[partA], 8, (255, 178, 102), thickness=-1, lineType=cv2.FILLED)
            cv2.circle(frame, points[partB], 8, (255, 178, 102), thickness=-1, lineType=cv2.FILLED)
    # Draw skeleton between left points
    for pair in LEFT_POSE_PAIRS:
        partA = pair[0]
        partB = pair[1]

        if points[partA] and points[partB]:
            # CHANGE THESE LINES TO DESIGN SKELETON
            cv2.line(frame, points[partA], points[partB], (255, 153, 204), 30, lineType=cv2.LINE_AA)
            cv2.circle(frame, points[partA], 8, (255, 102, 178), thickness=-1, lineType=cv2.FILLED)
            cv2.circle(frame, points[partB], 8, (255, 102, 178), thickness=-1, lineType=cv2.FILLED)
    # Add text to frame
    cv2.putText(frame, "hey there sexy you're looking good ;)", (50, 50), cv2.FONT_HERSHEY_COMPLEX, .8, (255, 204, 153), 2, lineType=cv2.LINE_AA)

    # Show frame
    cv2_imshow(frame)

    # Write frame to video
    vid_writer.write(frame)

vid_writer.release()

OLD MOTION DETECTION

In [ ]:
frameCount = 0
firstFrame = None
# loop over the frames of the video
while True:
    frameCount+=1
    # grab the current frame
    frame = vs.read()
    frame = frame[1]
    text = "Unoccupied"
    # if the frame could not be grabbed, then we have reached the end
	# of the video
    if frame is None:
        break
    # resize the frame, convert it to grayscale, and blur it
    frame = imutils.resize(frame,width=500)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray,(21,21),0)
    # if the first frame is None, initialize it
    if firstFrame is None:
        firstFrame = gray
        continue
    # compute the absolute difference between the current frame and
	# first frame
    frameDelta = cv2.absdiff(firstFrame, gray)
    thresh = cv2.threshold(frameDelta, 25, 255, cv2.THRESH_BINARY)[1]
	# dilate the thresholded image to fill in holes, then find contours
	# on thresholded image
    thresh = cv2.dilate(thresh, None, iterations=2)
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,
		cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
	# loop over the contours
    for c in cnts:
		# if the contour is too small, ignore it
        if cv2.contourArea(c) < 500:
            continue
		# compute the bounding box for the contour, draw it on the frame,
		# and update the text
        (x, y, w, h) = cv2.boundingRect(c)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        text = "Dancer!"
    # draw the text and timestamp on the frame
    #cv2.putText(frame, "Room Status: {}".format(text), (10, 20),
	#cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
    #cv2.putText(frame, datetime.datetime.now().strftime("%A %d %B %Y %I:%M:%S%p"),
	#	(10, frame.shape[0] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 0, 255), 1)
	# show the frame and record if the user presses a key
    #cv2_imshow(frame)
    #cv2_imshow(thresh)
    cv2_imshow(frameDelta)
    #key = cv2.waitKey(1) & 0xFF
    # if the `q` key is pressed, break from the lop
    #if key == ord("q"):
    #    break
# cleanup the camera and close any open windows
#vs.release()
#cv2.destroyAllWindows()
print(frameCount)